In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


## Initialize a nvcc plugin for python notebook

In [2]:
!pip install nvcc4jupyter

## Load the plugin extension

In [4]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


# Parallel Matrix multiplication version 1

In [35]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>

#define PRINT_LOG false // Toggle for logging matrices and intermediate outputs

// Kernel for 2D convolution using shared memory without tiling
__global__ void convolution_2D_shared_no_tiling(unsigned char *in, unsigned char *mask, unsigned char *out, int maskwidth, int w, int h) {
    // Declare dynamically allocated shared memory
    extern __shared__ unsigned char shared_mem[];
    unsigned char *shared_in = shared_mem; // Shared memory for input
    unsigned char *shared_mask = shared_mem + (blockDim.y + maskwidth - 1) * (blockDim.x + maskwidth - 1); // Shared memory for kernel/mask

    int tx = threadIdx.x; // Thread index in block (x-dimension)
    int ty = threadIdx.y; // Thread index in block (y-dimension)
    int Row = blockIdx.y * blockDim.y + ty; // Global row index
    int Col = blockIdx.x * blockDim.x + tx; // Global column index

    // Load kernel/mask into shared memory
    if (ty < maskwidth && tx < maskwidth) {
        shared_mask[ty * maskwidth + tx] = mask[ty * maskwidth + tx];
    }

    // Load the input matrix into shared memory with boundary checks
    int input_row = Row - maskwidth / 2;
    int input_col = Col - maskwidth / 2;

    if (input_row >= 0 && input_row < h && input_col >= 0 && input_col < w) {
        shared_in[ty * (blockDim.x + maskwidth - 1) + tx] = in[input_row * w + input_col];
    } else {
        shared_in[ty * (blockDim.x + maskwidth - 1) + tx] = 0; // Handle out-of-bounds access
    }

    __syncthreads(); // Ensure all threads have loaded data into shared memory

    // Perform convolution
    int pixVal = 0; // Accumulator for pixel value
    if (Row < h && Col < w) { // Check boundaries for valid computation
        for (int j = 0; j < maskwidth; ++j) {
            for (int k = 0; k < maskwidth; ++k) {
                pixVal += shared_in[(ty + j) * (blockDim.x + maskwidth - 1) + (tx + k)] * shared_mask[j * maskwidth + k];
            }
        }
        out[Row * w + Col] = (unsigned char)(pixVal); // Store result in output matrix
    }
}

// Kernel for 2D convolution using shared memory with tiling
__global__ void convolution_2D_shared_with_tiling(unsigned char *in, unsigned char *mask, unsigned char *out, int maskwidth, int w, int h) {
    extern __shared__ unsigned char shared_mem[];
    unsigned char *tile = shared_mem; // Shared memory for tile
    unsigned char *shared_mask = shared_mem + (blockDim.y + maskwidth - 1) * (blockDim.x + maskwidth - 1); // Shared memory for kernel/mask

    int tx = threadIdx.x; // Thread index in block (x-dimension)
    int ty = threadIdx.y; // Thread index in block (y-dimension)
    int Row = blockIdx.y * blockDim.y + ty; // Global row index
    int Col = blockIdx.x * blockDim.x + tx; // Global column index

    // Load kernel/mask into shared memory
    if (ty < maskwidth && tx < maskwidth) {
        shared_mask[ty * maskwidth + tx] = mask[ty * maskwidth + tx];
    }

    // Load the input tile into shared memory
    int input_row = Row - maskwidth / 2;
    int input_col = Col - maskwidth / 2;

    if (input_row >= 0 && input_row < h && input_col >= 0 && input_col < w) {
        tile[ty * (blockDim.x + maskwidth - 1) + tx] = in[input_row * w + input_col];
    } else {
        tile[ty * (blockDim.x + maskwidth - 1) + tx] = 0; // Handle out-of-bounds access
    }

    __syncthreads(); // Synchronize threads to ensure the tile is fully loaded

    // Perform convolution
    int pixVal = 0; // Accumulator for pixel value
    if (Row < h && Col < w) { // Check boundaries for valid computation
        for (int j = 0; j < maskwidth; ++j) {
            for (int k = 0; k < maskwidth; ++k) {
                pixVal += tile[(ty + j) * (blockDim.x + maskwidth - 1) + (tx + k)] * shared_mask[j * maskwidth + k];
            }
        }
        out[Row * w + Col] = (unsigned char)(pixVal); // Store result in output matrix
    }
}

int main() {
    // Variables for GPU properties
    int block_size;

    // Retrieve information about available CUDA devices
    int nDevices;
    cudaGetDeviceCount(&nDevices);
    for (int i = 0; i < nDevices; i++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, i);
        printf("Device Number: %d\n", i);
        printf("  Device name: %s\n", prop.name);
        printf("  max Blocks Per MultiProcessor: %d\n", prop.maxBlocksPerMultiProcessor);
        printf("  max Threads Per MultiProcessor: %d\n", prop.maxThreadsPerMultiProcessor);
        printf("  max Threads Per Block: %d\n", prop.maxThreadsPerBlock);
        printf("  num SM: %d\n", prop.multiProcessorCount);
        printf("  num bytes sharedMem Per Block: %d\n", prop.sharedMemPerBlock);
        printf("  num bytes sharedMem Per Multiprocessor: %d\n", prop.sharedMemPerMultiprocessor);
        printf("  Memory Clock Rate (KHz): %d\n", prop.memoryClockRate);
        printf("  Memory Bus Width (bits): %d\n", prop.memoryBusWidth);
        printf("  Peak Memory Bandwidth (GB/s): %f\n\n", 2.0 * prop.memoryClockRate * (prop.memoryBusWidth / 8) / 1.0e6);
    }

    // Define matrix and kernel sizes
    int matrix_sizes[] = {32, 128, 512, 1024, 2048, 4096, 8192, 10000};
    int mask_sizes[] = {3, 7, 15, 31, 63, 127};

    // Loop over matrix and kernel sizes
    for (int m_idx = 0; m_idx < sizeof(matrix_sizes) / sizeof(matrix_sizes[0]); ++m_idx) {
        for (int k_idx = 0; k_idx < sizeof(mask_sizes) / sizeof(mask_sizes[0]); ++k_idx) {
            int MATRIX_SIZE = matrix_sizes[m_idx];
            int MASK_WIDTH = mask_sizes[k_idx];
            int MATRIX_WITH, MATRIX_HEIGHT = matrix_sizes[m_idx];

            if (MASK_WIDTH >= MATRIX_SIZE) continue; // Skip cases where the kernel is larger than the matrix

            printf("\nRunning for MATRIX_SIZE = %d and MASK_WIDTH = %d\n", MATRIX_SIZE, MASK_WIDTH);

            // Allocate memory for host data
            unsigned char *a, *h_out, *h_mask;
            a = (unsigned char *)malloc(sizeof(unsigned char) * MATRIX_SIZE * MATRIX_SIZE);
            h_out = (unsigned char *)malloc(sizeof(unsigned char) * MATRIX_SIZE * MATRIX_SIZE);
            h_mask = (unsigned char *)malloc(sizeof(unsigned char) * MASK_WIDTH * MASK_WIDTH);

            if (PRINT_LOG) {
                // Print the kernel/mask
                printf("\nMask:  \n");
                for (int i = 0; i < MASK_WIDTH; ++i) {
                    printf("[");
                    for (int j = 0; j < MASK_WIDTH; ++j) {
                        h_mask[i * MASK_WIDTH + j] = 2;
                        printf(" %d ", h_mask[i * MASK_WIDTH + j]);
                    }
                    printf("]\n");
                }
            }

            if (PRINT_LOG) {
                // Print the input matrix
                printf("\nMatrix:  \n");
                for (int i = 0; i < MATRIX_SIZE; ++i) {
                    printf("[");
                    for (int j = 0; j < MATRIX_SIZE; ++j) {
                        a[i * MATRIX_SIZE + j] = 1;
                        printf(" %d ", a[i * MATRIX_SIZE + j]);
                    }
                    printf("]\n");
                }
            }

            // Allocate memory on the GPU
            unsigned char *d_in, *d_mask, *d_out;
            cudaMalloc((void **)&d_in, sizeof(unsigned char) * MATRIX_SIZE * MATRIX_SIZE);
            cudaMalloc((void **)&d_mask, sizeof(unsigned char) * MASK_WIDTH * MASK_WIDTH);
            cudaMalloc((void **)&d_out, sizeof(unsigned char) * MATRIX_SIZE * MATRIX_SIZE);

            // Copy input data to device
            cudaMemcpy(d_in, a, sizeof(unsigned char) * MATRIX_SIZE * MATRIX_SIZE, cudaMemcpyHostToDevice);
            cudaMemcpy(d_mask, h_mask, sizeof(unsigned char) * MASK_WIDTH * MASK_WIDTH, cudaMemcpyHostToDevice);

            // Configure thread and block dimensions
            int block_x = 16;
            int block_y = 16;
            dim3 dimBlock(block_x, block_y);
            dim3 dimGrid((MATRIX_SIZE + block_x - 1) / block_x, (MATRIX_SIZE + block_y - 1) / block_y, 1);

            // Measure elapsed time for the no-tiling kernel
            float elapsed_time_ms;
            cudaEvent_t start, stop;
            cudaEventCreate(&start);
            cudaEventCreate(&stop);

            cudaEventRecord(start, 0);
            convolution_2D_shared_no_tiling<<<dimGrid, dimBlock, (block_x + MASK_WIDTH - 1) * (block_y + MASK_WIDTH - 1) + MASK_WIDTH * MASK_WIDTH>>>(d_in, d_mask, d_out, MASK_WIDTH, MATRIX_SIZE, MATRIX_SIZE);
            cudaDeviceSynchronize();
            cudaEventRecord(stop, 0);
            cudaEventSynchronize(stop);
            cudaMemcpy(h_out, d_out, sizeof(unsigned char) * MATRIX_SIZE * MATRIX_SIZE, cudaMemcpyDeviceToHost);

            if (PRINT_LOG) {
                // Print the output matrix for no-tiling
                printf("\nOutput Matrix (No Tiling):\n");
                for (int i = 0; i < MATRIX_SIZE; ++i) {
                    printf("[");
                    for (int j = 0; j < MATRIX_SIZE; ++j) {
                        printf(" %d ", h_out[i * MATRIX_SIZE + j]);
                    }
                    printf("]\n");
                }
            }

            cudaEventElapsedTime(&elapsed_time_ms, start, stop);
            printf("\nconvolution_2D_shared_no_tiling Time elapsed: %f ms\n", elapsed_time_ms);

            // Measure elapsed time for the tiling kernel
            float elapsed_time_tiling_ms;
            cudaEvent_t start_tiling, stop_tiling;
            cudaEventCreate(&start_tiling);
            cudaEventCreate(&stop_tiling);

            cudaEventRecord(start_tiling, 0);
            convolution_2D_shared_with_tiling<<<dimGrid, dimBlock, (block_x + MASK_WIDTH - 1) * (block_y + MASK_WIDTH - 1) + MASK_WIDTH * MASK_WIDTH>>>(d_in, d_mask, d_out, MASK_WIDTH, MATRIX_SIZE, MATRIX_SIZE);
            cudaDeviceSynchronize();
            cudaEventRecord(stop_tiling, 0);
            cudaEventSynchronize(stop_tiling);
            cudaMemcpy(h_out, d_out, sizeof(unsigned char) * MATRIX_SIZE * MATRIX_SIZE, cudaMemcpyDeviceToHost);

            if (PRINT_LOG) {
                // Print the output matrix for tiling
                printf("\nOutput Matrix (With Tiling):\n");
                for (int i = 0; i < MATRIX_SIZE; ++i) {
                    printf("[");
                    for (int j = 0; j < MATRIX_SIZE; ++j) {
                        printf(" %d ", h_out[i * MATRIX_SIZE + j]);
                    }
                    printf("]\n");
                }
            }

            cudaEventElapsedTime(&elapsed_time_tiling_ms, start_tiling, stop_tiling);
            printf("\nconvolution_2D_shared_with_tiling Time elapsed: %f ms\n", elapsed_time_tiling_ms);

            // Free GPU memory
            cudaFree(d_in);
            cudaFree(d_mask);
            cudaFree(d_out);
            // Free host memory
            free(a);
            free(h_out);
            free(h_mask);
        }
    }

    return 0; // Exit the program
}

Device Number: 0
  Device name: Tesla T4
  max Blocks Per MultiProcessor: 16
  max Threads Per MultiProcessor: 1024
  max Threads Per Block: 1024
  num SM: 40
  num bytes sharedMem Per Block: 49152
  num bytes sharedMem Per Multiprocessor: 65536
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064000


Running for MATRIX_SIZE = 32 and MASK_WIDTH = 3

convolution_2D_shared_no_tiling Time elapsed: 0.233472 ms

convolution_2D_shared_with_tiling Time elapsed: 0.026880 ms

Running for MATRIX_SIZE = 32 and MASK_WIDTH = 7

convolution_2D_shared_no_tiling Time elapsed: 0.020320 ms

convolution_2D_shared_with_tiling Time elapsed: 0.026112 ms

Running for MATRIX_SIZE = 32 and MASK_WIDTH = 15

convolution_2D_shared_no_tiling Time elapsed: 0.028640 ms

convolution_2D_shared_with_tiling Time elapsed: 0.029312 ms

Running for MATRIX_SIZE = 32 and MASK_WIDTH = 31

convolution_2D_shared_no_tiling Time elapsed: 0.067968 ms

convolution_2D_shared_with